## Setup

In [1]:
import gc
import re
import os
import json 
import math
import shutil
import random
import warnings
from os.path import join
from functools import partial
from tqdm.notebook import tqdm
from collections import defaultdict
from operator import methodcaller
from typing import Optional, Literal
from typing import Optional, Literal, Iterator
from itertools import pairwise, starmap, product

import torch
import optuna
import kagglehub 
import numpy as np
import pandas as pd
import polars as pl
from numpy import ndarray
from torch import nn, Tensor
from numpy.linalg import norm
import torch.nn.functional as F
from torch.optim import Optimizer
from pandas import DataFrame as DF
from optuna.trial import TrialState
from sklearn.metrics import f1_score
from optuna.pruners import BasePruner
from optuna.exceptions import TrialPruned
from torch.utils.data import TensorDataset
from scipy.spatial.transform import Rotation
import kaggle_evaluation.cmi_inference_server
from torch.utils.data import DataLoader as DL
from sklearn.model_selection import GroupKFold
from rich.progress import Progress, Task, track
from sklearn.model_selection import train_test_split
from numpy.lib.stride_tricks import sliding_window_view
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.utils.class_weight import compute_class_weight
from torch.optim.lr_scheduler import ConstantLR, LRScheduler, _LRScheduler

from config import *
from model import mk_model
from training import CMIDataset

In [2]:
GATING_INPUT_FEATURES = [
    "bin_mae",
    "reg_mae",
    "y_uncertainty",
    "bin_uncertainty",
    "reg_uncertainty",
    "orient_uncertainty",
]
GATING_MODEL_BATCH_SIZE = 256
N_GATING_MODEL_EPOCHS = 4

## Data

In [ ]:
def record_target_feature(metrics:defaultdict, y_pred:Tensor, y_true:Tensor, preffix:str):
    y_pred = y_pred.cpu().numpy()
    y_true = y_true.cpu().numpy()
    for target_col_idx in range(y_pred.shape[1]):
        metrics[preffix + "_pred_" + str(target_col_idx)].append(y_pred[:, target_col_idx])
        metrics[preffix + "_true_" + str(target_col_idx)].append(y_true[:, target_col_idx])

def get_perf_and_seq_id(model:nn.Module, data_loader:DL, device:torch.device, seq_meta_data:DF) -> DF:
    metrics:dict[list[ndarray]] = defaultdict(list)
    model = model.eval()
    with torch.no_grad():
        for batch_x, batch_y, batch_orient_y, batch_bin_demos_y, batch_reg_demos_y, idx in data_loader:
            batch_x = batch_x.to(device).clone()
            batch_y = batch_y.to(device)

            outputs, orient_outputs, bin_demos_output, reg_demos_output = model(batch_x)
            losses = nn.functional.cross_entropy(
                outputs,
                batch_y,
                label_smoothing=LABEL_SMOOTHING,
                reduction="none",
            )
            orient_losses = nn.functional.cross_entropy(
                orient_outputs,
                batch_orient_y,
                label_smoothing=LABEL_SMOOTHING,
                reduction="none",
            )
            bin_demos_losses = nn.functional.binary_cross_entropy_with_logits(
                bin_demos_output,
                batch_bin_demos_y,
                # label_smoothing=LABEL_SMOOTHING,
                reduction="none",
            ).cpu().numpy()
            reg_demos_losses = nn.functional.mse_loss(reg_demos_output, batch_reg_demos_y, reduction="none").cpu().numpy()
            metrics["losses"].append(losses.cpu().numpy())
            metrics["orient_losses"].append(orient_losses.cpu().numpy())
            record_target_feature(metrics, outputs, batch_y, "y")
            record_target_feature(metrics, orient_outputs, batch_orient_y, "orient")
            record_target_feature(metrics, bin_demos_output, batch_bin_demos_y, "bin")
            record_target_feature(metrics, batch_reg_demos_y, reg_demos_output, "reg")
            metrics["sequence_id"].append(seq_meta_data["sequence_id"].iloc[idx].values)

    metrics = {k: np.concat(v) for k, v in metrics.items()}

    return DF(metrics)

In [4]:
EPSILON = 1e-12

def preds_uncertainty(df:DF, task_preffix:str) -> pd.Series:
    preds = df.filter(regex=f"{task_preffix}_pred_*", axis="columns")
    clipped_preds = preds.clip(EPSILON, 1.0)
    df[task_preffix + "_uncertainty"] = -((clipped_preds * np.log(clipped_preds)).sum(axis=1))
    return df

def mae(df:DF, task_preffix:str) -> DF:
    df_pred = df.filter(regex=f"{task_preffix}_pred_*", axis="columns")
    df_true = df.filter(regex=f"{task_preffix}_true_*", axis="columns")
    df[task_preffix + "_mae"] = np.abs(df_pred.values - df_true.values).mean(axis=1)

    return df

def post_process_df(df:DF) -> DF:
    return (
        df
        .pipe(preds_uncertainty, "y")
        .pipe(preds_uncertainty, "orient")
        .pipe(preds_uncertainty, "bin")
        .pipe(preds_uncertainty, "reg")
        .pipe(mae, "y")
        .pipe(mae, "orient")
        .pipe(mae, "bin")
        .pipe(mae, "reg")
    )

def record_models_outputs() -> DF:
    device = torch.device("cuda")
    dataset = CMIDataset(device)
    data_loader = DL(dataset, batch_size=1024, shuffle=False)
    seq_meta_data = pd.read_parquet("preprocessed_dataset/sequences_meta_data.parquet")
    dfs = []
    for fold_idx in tqdm(range(N_FOLDS), total=N_FOLDS):
        model = mk_model(device=device)
        checkpoint = torch.load(
            join(
                "models",
                f"model_fold_{fold_idx}.pth"
            ),
            map_location=device,
            weights_only=True
        )
        model.load_state_dict(checkpoint)
        model.eval()
        df = get_perf_and_seq_id(model, data_loader, device, seq_meta_data)
        dfs.append(df.assign(fold=fold_idx))

    return pd.concat(dfs).pipe(post_process_df)

In [5]:
df = record_models_outputs()

  0%|          | 0/20 [00:00<?, ?it/s]

## Training

In [7]:
def get_gating_features(df:DF) -> ndarray:
    return (
        df
        .loc[:, GATING_INPUT_FEATURES + ["fold"]]
        .pivot(columns="fold")
        .values
    )

def get_experts_outputs(df:DF) -> ndarray:
    return (
        df
        .filter(regex=r"^y_pred_|^fold$")
        .pivot(columns="fold")
        .values
        .reshape(-1, N_FOLDS, N_TARGETS)
    )

def get_gating_targets(df:DF) -> ndarray:
    return (
        df
        .query("fold == 0")
        .filter(regex="y_true*")
        .values
    )

class GatingDataset(TensorDataset):
    def __init__(self, df: DF, device: torch.device):
        super().__init__(
            torch.from_numpy(get_gating_features(df)).to(device),
            torch.from_numpy(get_experts_outputs(df)).to(device),
            torch.from_numpy(get_gating_targets(df)).to(device),
        )

In [52]:
def evaluate_gating_model(dataset: Dataset, gating_model: nn.Module) -> dict:
    gating_model = gating_model.eval()
    data_loader = DL(dataset, batch_size=1024, shuffle=False)
    y_preds = []
    with torch.no_grad():
        for x, experts_y_preds, _ in data_loader:
            weights = gating_model(x)
            y_pred = torch.einsum("be, bet -> bt", weights, experts_y_preds) / weights.sum(dim=1, keepdim=True)
            y_preds.append(y_pred)

    y_preds = torch.argmax(torch.concat(y_preds), dim=1)
    y_true = torch.argmax(dataset.tensors[2], dim=1)
    metrics = {
        "accuracy": (y_preds == y_true).float().mean(),
    }

    return metrics

def train_model_on_single_epoch(data_loader: DL, gating_model: nn.Module, criterion: nn.Module, optimizer: Optimizer) -> dict:
    metrics = defaultdict(float)
    n_samples = 0
    gating_model = gating_model.train()
    for x, experts_y_preds, y_true in data_loader:
        optimizer.zero_grad()
        weights = gating_model(x)
        y_pred = torch.einsum("be, bet -> bt", weights, experts_y_preds) / weights.sum(dim=1, keepdim=True)

        loss = criterion(y_pred, y_true)
        loss.backward()
        optimizer.step()
        n_samples += x.shape[0]
        metrics["train_loss"] += loss.item() * x.shape[0]

    metrics["train_loss"] /= n_samples
    
    return metrics

def train_model_on_all_epochs(dataset: Dataset, gating_model: nn.Module) -> DF:
    train_loader = DL(dataset, GATING_MODEL_BATCH_SIZE, shuffle=True)
    optimizer = torch.optim.AdamW(gating_model.parameters())
    metrics: list[dict] = []
    criterion = nn.CrossEntropyLoss()
    for epoch in range(N_GATING_MODEL_EPOCHS):
        train_metrics = train_model_on_single_epoch(train_loader, gating_model, criterion, optimizer)
        eval_metrics = evaluate_gating_model(dataset, gating_model)
        metrics.append({"epoch": epoch} | train_metrics | eval_metrics)

    return DF.from_records(metrics)

In [44]:
class MeanGate(nn.Module):
    def ___init__(self, device: torch.device):
        self.device = device
    
    def forward(self, experts_preds_stats: Tensor) -> Tensor:
        return torch.ones((experts_preds_stats.shape[0], N_FOLDS)).cuda()
    
class LogisticRegression(nn.Sequential):
    def __init__(self):
        super().__init__(
            nn.LazyLinear(N_FOLDS),
            nn.Sigmoid(),
        )

In [ ]:
device = torch.device("cuda")
dataset = GatingDataset(df, device)
mean_gate_metrics = evaluate_gating_model(dataset, MeanGate())
print("mean_gate_metrics:", mean_gate_metrics)
logisticGate = LogisticRegression().to(device)
base_logistic_gate_metrics = evaluate_gating_model(dataset, logisticGate)
print("base logistic_gate_metrics:", base_logistic_gate_metrics)
training_metrics = train_model_on_all_epochs(dataset, logisticGate)
print("trained logistic_gate_metrics:", training_metrics.iloc[-1])

mean_gate_metrics: {'accuracy': tensor(0.0518, device='cuda:0')}
base logistic_gate_metrics: {'accuracy': tensor(0.0703, device='cuda:0')}
trained logistic_gate_metrics: epoch                                       3
train_loss                           2.737511
accuracy      tensor(0.1554, device='cuda:0')
Name: 3, dtype: object
